In [4]:
import requests
from bs4 import BeautifulSoup
import re
import sys
import pandas as pd
import numpy as np
import pickle
import string

In [71]:
LETTERS = string.ascii_uppercase

In [76]:
def scrape_streets():
    LETTERS = string.ascii_uppercase
    streets = []
    localities = []
    for letter in LETTERS:
        path = f'https://berlin.kauperts.de/Strassenverzeichnis/{letter}'
        response = requests.get(path)
        page_html = response.text
        page_soup = BeautifulSoup(page_html)
        table_soup = page_soup.find(attrs={'class': 'default streets'}).find_all('td')
        for i, line in enumerate(table_soup):
            if i % 2 == 0: # it's a street
                streets.append(line.text)
            else: # it's an Ortsteil
                localities.append(line.text)
    streets_dict = {'Straße': streets, 'Ortsteile': localities}
    return streets_dict

CPU times: user 5.86 s, sys: 268 ms, total: 6.13 s
Wall time: 16.6 s


In [ ]:
streets_dict = scrape_parks()

In [255]:
# pickle streets_dict
with open('berlin_streets_dict.pickle', 'wb') as f:
    pickle.dump(streets_dict, f)

In [330]:
with open('berlin_streets_dict.pickle', 'rb') as f:
    loaded_streets_dict = pickle.load(f)

In [331]:
streets_dict == loaded_streets_dict

True

In [319]:
def scrape_parks():
    LETTERS = string.ascii_uppercase
    parks = []
    park_locations = []
    park_PLZ = []
    for letter in LETTERS:
        path = f'https://berlin.kauperts.de/Adressen/Gruenanlage/{letter}'
        response = requests.get(path)
        page_html = response.text
        page_soup = BeautifulSoup(page_html)
        try:
            table_soup = page_soup.find(attrs={'class': 'default providers'}).find_all('td')
        except:
            continue
        for i, cell in enumerate(table_soup):
            if i % 3 == 0:
                parks.append(cell.a.text)
            elif i % 3 == 1:
                location, plz = re.findall(' +(.*)\\xa0–\\n *(\d{5})', cell.string)[0]
                park_locations.append(location)
                park_PLZ.append(plz)
            else:
                continue
    parks_dict = {'Park': parks, 'Ortsteil': park_locations, 'PLZ': park_PLZ}
    return parks_dict

In [320]:
parks_dict = scrape_parks()

In [322]:
# pickle parks_dict
with open('berlin_parks_dict.pickle', 'wb') as f:
    pickle.dump(parks_dict, f)

In [325]:
with open('berlin_parks_dict.pickle', 'rb') as f:
    loaded_parks_dict = pickle.load(f)

In [326]:
parks_dict == loaded_parks_dict

True